In [ ]:
!pip install transformers peft datasets bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.1 MB/s eta 0:00:00


In [3]:
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.2 MB/s eta 0:00:00


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

In [9]:
!pip install -U bitsandbytes

In [2]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
dataset = load_dataset("Abirate/english_quotes")  # example dataset

tokenizer = AutoTokenizer.from_pretrained(model_name)


import torch
from transformers import BitsAndBytesConfig

config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", quantization_config=config)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [10]:
def combine_fields(example):
    text = f"{example['quote']} — {example['author']}"
    return {"text": text}

dataset = dataset.map(combine_fields)


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [11]:
dataset["train"][0]["text"]


'“Be yourself; everyone else is already taken.” — Oscar Wilde'

In [22]:
def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )


In [23]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Now map works fine
tokenized_dataset = dataset.map(
    tokenize,
    batched=True,
    remove_columns=dataset["train"].column_names
)


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [24]:
tokenized_dataset = tokenized_dataset.map(
    lambda batch: {"labels": batch["input_ids"]},
    batched=True
)


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [26]:
args = TrainingArguments(
    output_dir="mistral-7b-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    fp16=True,
    learning_rate=2e-4,
    remove_unused_columns=False,
    num_train_epochs=1,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"]
)

trainer.train()

model.save_pretrained("mistral_lora_adapter")

Step,Training Loss


In [36]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

# This makes Mistral-7B fit in ~6GB VRAM instead of 14GB
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",   # or mistralai/ if your adapter matches
    quantization_config=quantization_config,
    device_map="auto",           # automatically puts layers on GPU
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token = tokenizer.eos_token  # important for batching

# Load and merge your LoRA adapter
model = PeftModel.from_pretrained(base_model, "mistral_lora_adapter")
model = model.merge_and_unload()  # now merged, no extra memory

print("Model loaded and merged successfully!")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/peft/peft_model.py:585: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.2.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.2.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.2.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.2.self_attn.v_proj.l

Model loaded and merged successfully!


In [35]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# This one line fixes your exact error
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto",
    offload_folder="offload_weights",   # creates folder automatically
    offload_state_dict=True,            # extra safety flag
    dtype="auto"
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

model = PeftModel.from_pretrained(base_model, "mistral_lora_adapter")
model = model.merge_and_unload()  # optional: get a normal model back

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 2.12 MiB is free. Process 15861 has 14.74 GiB memory in use. Of the allocated memory 14.22 GiB is allocated by PyTorch, and 387.05 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [42]:
prompt = "Be who you are"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate output
outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    temperature=0.1,
    top_p=0.9,
    do_sample=True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Be who you are and say what you feel, because those who mind don’t matter and those who matter don’t mind.

I’ve always loved this quote by Dr. Seuss. It’s a reminder to be true to ourselves and not


In [43]:
# After you have the merged model (exactly where you are now)
merged_model_path = "./my_mistral7b_finetuned_merged"

model.save_pretrained(merged_model_path)           # Saves the full merged model
tokenizer.save_pretrained(merged_model_path)       # Saves tokenizer + special tokens

print(f"Model permanently saved to {merged_model_path}")
print("Next time just load from this folder — 100% offline!")

Model permanently saved to ./my_mistral7b_finetuned_merged
Next time just load from this folder — 100% offline!
